In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from os import makedirs
from os.path import join, exists, expanduser
from keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions

In [2]:
img_des = pd.read_csv('D:\RMBI_MMKG_data\img_des.csv')
img_des

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
14179,/m/0dlj8q2,Le Monde's 100 Books of the Century,The 100 Books of the Century is a list of the ...
14180,/m/05zp8,Palace,"A palace is a grand residence, especially a ro..."
14181,/m/024030,Padma Vibhushan,The Padma Vibhushan is the second-highest civi...
14182,/m/02y_9cf,Ethiopian Empire,"The Ethiopian Empire, also formerly known by t..."


In [3]:
SIZE = 1000

In [4]:
img_des_trunc = img_des.head(SIZE)
img_des_trunc

,LabelName,name,description
0,/m/027rn,Dominican Republic,The Dominican Republic is a country located on...
1,/m/017dcd,Mighty Morphin Power Rangers,Mighty Morphin Power Rangers is an American su...
2,/m/01sl1q,Michelle Rodriguez,Mayte Michelle Rodriguez is an American actres...
3,/m/0cnk2q,Australia national football team,The Australia men's national soccer team repre...
4,/m/02_j1w,Defender,"In the sport of association football, a defend..."
...,...,...,...
995,/m/01b9ck,Darryl F. Zanuck,Darryl Francis Zanuck was an American film pro...
996,/m/011ywj,Gosford Park,Gosford Park is a 2001 satirical black comedy ...
997,/m/0325pb,Alpha Delta Pi,"Alpha Delta Pi, commonly known as ADPi, is an ..."
998,/m/0mnrb,Henrico County,"Henrico County, officially the County of Henri..."


In [5]:
text_vec = pd.DataFrame(columns=['LabelName','name','title_vec','des_vec'])
text_vec['LabelName'] = img_des_trunc['LabelName']
text_vec['name'] = img_des_trunc['name']
text_vec['title_vec']=text_vec['title_vec'].astype(object)
text_vec['des_vec']=text_vec['des_vec'].astype(object)
text_vec

,LabelName,name,title_vec,des_vec
0,/m/027rn,Dominican Republic,NaN,NaN
1,/m/017dcd,Mighty Morphin Power Rangers,NaN,NaN
2,/m/01sl1q,Michelle Rodriguez,NaN,NaN
3,/m/0cnk2q,Australia national football team,NaN,NaN
4,/m/02_j1w,Defender,NaN,NaN
...,...,...,...,...
995,/m/01b9ck,Darryl F. Zanuck,NaN,NaN
996,/m/011ywj,Gosford Park,NaN,NaN
997,/m/0325pb,Alpha Delta Pi,NaN,NaN
998,/m/0mnrb,Henrico County,NaN,NaN


In [6]:
import sister
sentence_embedding = sister.BertEmbedding(lang="en")

for i in range(SIZE):
    if (pd.notnull(img_des_trunc['name'][i])):
        text_vec['title_vec'][i] = sentence_embedding(img_des_trunc['name'][i])
    if (pd.notnull(img_des_trunc['description'][i])):
        text_vec['des_vec'][i] = sentence_embedding(img_des_trunc['description'][i]) 
text_vec

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\Alex\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the l

,LabelName,name,title_vec,des_vec
0,/m/027rn,Dominican Republic,"[0.4945737, 1.3592469, 1.282383, 0.01269114, -...","[0.7342588, -0.70393395, 1.6656038, 0.67461014..."
1,/m/017dcd,Mighty Morphin Power Rangers,"[1.6969184, 1.8559471, 0.8122708, -0.40268016,...","[0.025732398, -0.7696621, 2.172137, 1.2750009,..."
2,/m/01sl1q,Michelle Rodriguez,"[1.2370641, 2.8018966, 0.16598952, -0.5138393,...","[0.6779196, -0.15636724, 0.9539355, 0.6223599,..."
3,/m/0cnk2q,Australia national football team,"[-0.34985536, 1.8396395, 1.8012114, -0.3399124...","[-0.7440083, 0.010402456, 2.7207577, 0.6726733..."
4,/m/02_j1w,Defender,"[0.10609877, 1.8506274, 1.07485, -0.12472837, ...","[-0.1949073, -0.38760465, 3.056767, 0.1185953,..."
...,...,...,...,...
995,/m/01b9ck,Darryl F. Zanuck,"[1.229999, 2.4127681, 0.94314957, -1.3280284, ...","[0.73823285, -0.08005962, 0.827093, 0.8234503,..."
996,/m/011ywj,Gosford Park,"[0.7568916, 1.5736562, 0.8222938, -0.13049412,...","[0.26722866, -0.9173725, 1.927658, 0.5772505, ..."
997,/m/0325pb,Alpha Delta Pi,"[0.5692497, 1.3098, 1.3415865, -0.55102956, -0...","[0.18281287, -0.25513795, 2.1837523, 0.3496461..."
998,/m/0mnrb,Henrico County,"[0.059751034, 0.558505, 0.9079651, -0.01371228...","[-0.95366466, -0.1571609, 1.9237427, -0.214625..."


In [7]:
text_vec['des_vec']

0      [0.7342588, -0.70393395, 1.6656038, 0.67461014...
1      [0.025732398, -0.7696621, 2.172137, 1.2750009,...
2      [0.6779196, -0.15636724, 0.9539355, 0.6223599,...
3      [-0.7440083, 0.010402456, 2.7207577, 0.6726733...
4      [-0.1949073, -0.38760465, 3.056767, 0.1185953,...
                             ...                        
995    [0.73823285, -0.08005962, 0.827093, 0.8234503,...
996    [0.26722866, -0.9173725, 1.927658, 0.5772505, ...
997    [0.18281287, -0.25513795, 2.1837523, 0.3496461...
998    [-0.95366466, -0.1571609, 1.9237427, -0.214625...
999    [0.7939751, 0.95126754, 0.88291603, 1.622468, ...
Name: des_vec, Length: 1000, dtype: object

In [8]:
y = []
x_img = []
x_title = []
x_des = []
import os

for i in range(SIZE):
    path_of_the_directory = os.path.join('D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader', img_des['name'][i])
    ext = ('.jpg')
    for path, dirc, files in os.walk(path_of_the_directory):
        for name in files:
            if name.endswith(ext):
                
                img_path = os.path.join(path, name)
                try:                
                    img = image.load_img(img_path, target_size=(224, 224))
                    img = image.img_to_array(img)
                
                    x_img.append(img)
                    y.append(img_des['name'][i])
                    x_title.append(text_vec['title_vec'][i])
                    x_des.append(text_vec['des_vec'][i])
                except:
                    print(img_path)

C:\Users\Alex\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\World War II\CG-lQt2UIAAMHnz.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.\DNPRuwzW4AEATin.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Crystal Palace F.C.\DXIZ3FtX0AARZs-.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\United Nations\C-SBcq9XYAACzDM.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Marvel Entertainment\BO_SLlbCQAAhW3o.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Billy Connolly\DqA-b32WsAAYAIU.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\APOEL FC\DBuISFIXsAYsLy-.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Fayetteville\DWaC5hiVQAIoGLU.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\SG Dynamo Dresden\DjliQpmXcAExxHJ.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Shrek\CG0LV7KWcAABHbU.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Dan Povenmire\DKC4jwsUQAAxtSs.jpg
D:\RMBI_MMKG_data\Bulk-Bing-Image-downloader\Election\CwtiPuWUkAEXeSY.jpg


In [35]:
y = np.array(y)
x_img = np.array(x_img)
x_title = np.array(x_title)
x_des = np.array(x_des)

In [36]:
print(y.shape)
print(x_img.shape)
print(x_title.shape)
print(x_des.shape)

(8493,)
(8493, 224, 224, 3)
(8493, 768)
(8493, 768)


In [37]:
y

array(['Dominican Republic', 'Dominican Republic', 'Dominican Republic',
       ..., 'Theatrical producer', 'Theatrical producer',
       'Theatrical producer'], dtype='<U97')

In [38]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y)

In [39]:
y_encoded

array([265, 265, 265, ..., 890, 890, 890])

In [40]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y_encoded)
y_one_hot

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [41]:
y_one_hot.shape

(8493, 983)

In [42]:
from sklearn.model_selection import train_test_split
x_img_train, x_img_test, x_title_train, x_title_test,x_des_train, x_des_test, y_one_hot_train, y_one_hot_test, y_encoded_train, y_encoded_test = train_test_split(x_img, x_title, x_des, y_one_hot, y_encoded, test_size=0.2)

In [43]:
x_img_train

array([[[[112., 110., 111.],
         [122., 120., 121.],
         [111., 109., 110.],
         ...,
         [220., 220., 220.],
         [223., 223., 225.],
         [223., 223., 225.]],

        [[184., 182., 183.],
         [197., 195., 196.],
         [206., 204., 205.],
         ...,
         [221., 221., 221.],
         [223., 223., 225.],
         [223., 223., 225.]],

        [[193., 191., 192.],
         [190., 188., 189.],
         [176., 174., 175.],
         ...,
         [231., 231., 231.],
         [223., 223., 225.],
         [223., 223., 225.]],

        ...,

        [[ 10.,  10.,  10.],
         [ 17.,  17.,  17.],
         [134., 134., 134.],
         ...,
         [160., 205., 172.],
         [148., 189., 155.],
         [202., 230., 207.]],

        [[  2.,   2.,   2.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         ...,
         [196., 224., 199.],
         [118., 155., 124.],
         [215., 249., 224.]],

        [[  3.,   3.,   3.],
       

In [44]:
x_img_train, x_img_test = x_img_train / 255.0, x_img_test / 255.0

In [45]:
x_img_train

array([[[[0.4392157 , 0.43137255, 0.43529412],
         [0.47843137, 0.47058824, 0.4745098 ],
         [0.43529412, 0.42745098, 0.43137255],
         ...,
         [0.8627451 , 0.8627451 , 0.8627451 ],
         [0.8745098 , 0.8745098 , 0.88235295],
         [0.8745098 , 0.8745098 , 0.88235295]],

        [[0.72156864, 0.7137255 , 0.7176471 ],
         [0.77254903, 0.7647059 , 0.76862746],
         [0.80784315, 0.8       , 0.8039216 ],
         ...,
         [0.8666667 , 0.8666667 , 0.8666667 ],
         [0.8745098 , 0.8745098 , 0.88235295],
         [0.8745098 , 0.8745098 , 0.88235295]],

        [[0.75686276, 0.7490196 , 0.7529412 ],
         [0.74509805, 0.7372549 , 0.7411765 ],
         [0.6901961 , 0.68235296, 0.6862745 ],
         ...,
         [0.90588236, 0.90588236, 0.90588236],
         [0.8745098 , 0.8745098 , 0.88235295],
         [0.8745098 , 0.8745098 , 0.88235295]],

        ...,

        [[0.03921569, 0.03921569, 0.03921569],
         [0.06666667, 0.06666667, 0.06666667]

In [46]:
ResNet = ResNet50(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
for layer in ResNet.layers:
    layer.trainable = False
ResNet = Model(inputs=ResNet.inputs, outputs=ResNet.layers[-2].output)
ResNet.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [47]:
features = ResNet.predict(x_img_train)
features_test = ResNet.predict(x_img_test)

In [48]:
features.shape

(6794, 2048)

In [49]:
img_input = tf.keras.Input(shape=(2048,), name='img_input')
title_input = tf.keras.Input(shape=(768,), name='title_input')
des_input = tf.keras.Input(shape=(768,), name='des_input')

img_feat = tf.keras.layers.Dense(768,)(img_input)

x = tf.keras.layers.Concatenate(axis=1)([img_input,title_input,img_feat])

name_pred = tf.keras.layers.Dense(y_one_hot.shape[1],name="name")(x)

model = tf.keras.Model(
    inputs=[img_input, title_input, des_input],
    outputs=[name_pred],
)

In [50]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-3),
    loss=[
        tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    ],
    metrics=['accuracy']
)

In [51]:
print("Fit model on training data")
history = model.fit(
    [features,x_title_train,x_des_train],
    y_one_hot_train,
    batch_size=20,
    epochs=10,
)

Fit model on training data
Epoch 1/10
340/340 [==============================] - 7s 20ms/step - loss: 6.2079 - accuracy: 0.0967
Epoch 2/10
340/340 [==============================] - 7s 20ms/step - loss: 2.7588 - accuracy: 0.4787
Epoch 3/10
340/340 [==============================] - 7s 20ms/step - loss: 1.6038 - accuracy: 0.6874
Epoch 4/10
340/340 [==============================] - 7s 20ms/step - loss: 1.1514 - accuracy: 0.7648
Epoch 5/10
340/340 [==============================] - 7s 20ms/step - loss: 0.8957 - accuracy: 0.8131
Epoch 6/10
340/340 [==============================] - 7s 20ms/step - loss: 0.7565 - accuracy: 0.8354
Epoch 7/10
340/340 [==============================] - 7s 21ms/step - loss: 0.6367 - accuracy: 0.8628
Epoch 8/10
340/340 [==============================] - 7s 20ms/step - loss: 0.5766 - accuracy: 0.8689
Epoch 9/10
340/340 [==============================] - 7s 20ms/step - loss: 0.5467 - accuracy: 0.8775
Epoch 10/10
340/340 [==============================] - 7s 20ms/s

In [52]:
results = model.evaluate(
    [features_test,x_title_test,x_des_test], 
    y_one_hot_test, 
    batch_size=10)

170/170 [==============================] - 0s 1ms/step - loss: 0.6644 - accuracy: 0.8534


In [53]:
from sklearn.metrics import classification_report

y_pred = model.predict([features_test,x_title_test,x_des_test], batch_size=10, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_encoded_test, y_pred_bool))

170/170 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         4
           2       0.13      1.00      0.23         3
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00         2
           7       1.00      0.25      0.40         4
           8       0.00      0.00      0.00         1
           9       0.75      1.00      0.86         3
          11       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         2
          17       0.50      1.00      0.67         1
          18       0.00      0.00      0.00         1
          19       1.00   

C:\Users\Alex\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Alex\anaconda3\lib\site-packages\sklearn\metrics\_classif

In [54]:
model.save('classification.h5')